# Data Engineer Assignment [Tridge]

2022.01.23

The attached dataset represents prices of kimchi during the first three months of 2018.

This dataset is identified by their specifications such as Date, Price, Total Volume, Total Boxes, Small Boxes, Large Boxes, XLarge Boxes, Region. 

* Q.1 [Code Submission optional] Apply your knowledge of data engineering to Clean and Transform raw data into relevant “Features”. You are free to define what Features can be meaningful from this dataset. Briefly add an explanation of the method you used, and why?


* Q.2 [Code submission required] Create a pipeline that receives a set of attributes and returns a filtered dataset matching those attributes. (ex: Date / Price / Region)

In [1]:
from functools import partial
import pandas as pd

In [2]:
df = pd.read_excel('Kimchi_dataset.xlsx')
df.head()

,Date,Price,Total Volume,Total Boxes,Small Boxes,Large Boxes,XLarge Boxes,Region
0,2018-03-25,1.71,2321.82,2006.46,1996.46,10.00,0.0,Seoul
1,2018-03-18,1.66,3154.45,2580.60,2577.27,3.33,0.0,Seoul
2,2018-03-11,1.68,2570.52,2209.29,2209.29,0.00,0.0,Seoul
3,2018-03-04,1.48,3851.30,3242.98,3239.65,3.33,0.0,Seoul
4,2018-02-25,1.56,5356.63,4007.48,4007.48,0.00,0.0,Seoul


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 648 entries, 0 to 647
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          648 non-null    datetime64[ns]
 1   Price         644 non-null    float64       
 2   Total Volume  647 non-null    float64       
 3   Total Boxes   648 non-null    float64       
 4   Small Boxes   648 non-null    float64       
 5   Large Boxes   648 non-null    float64       
 6   XLarge Boxes  648 non-null    float64       
 7   Region        648 non-null    object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 40.6+ KB


In [4]:
df.isnull().sum()

Date            0
Price           4
Total Volume    1
Total Boxes     0
Small Boxes     0
Large Boxes     0
XLarge Boxes    0
Region          0
dtype: int64

In [5]:
df.describe()

,Price,Total Volume,Total Boxes,Small Boxes,Large Boxes,XLarge Boxes
count,644.000000,6.470000e+02,6.480000e+02,648.000000,648.000000,648.000000
mean,6.533540,7.500591e+04,4.963935e+04,40598.748580,9032.257948,8.345370
std,118.516792,2.127440e+05,1.407772e+05,115044.249219,29633.321649,43.304003
min,1.010000,2.064900e+03,9.884500e+02,284.430000,0.000000,0.000000
25%,1.410000,9.168750e+03,6.724712e+03,4813.205000,14.322500,0.000000
50%,1.550000,1.772227e+04,1.186087e+04,9120.540000,622.720000,0.000000
75%,1.710000,4.117210e+04,2.915701e+04,25105.742500,3289.035000,0.000000
max,3003.000000,1.814930e+06,1.124622e+06,944572.500000,267818.310000,510.410000


In [6]:
df.loc[df['Price'].isna()]

,Date,Price,Total Volume,Total Boxes,Small Boxes,Large Boxes,XLarge Boxes,Region
38,2018-03-11,NaN,NaN,2422.18,664.03,1758.15,0.0,deagu
246,2018-02-11,NaN,5216.61,4208.15,2420.23,1787.92,0.0,Gongju
511,2018-02-04,NaN,28070.07,2202.21,2202.21,0.00,0.0,Gyeongju
641,2018-02-18,NaN,17597.12,13776.71,13553.53,223.18,0.0,Boryeong


In [7]:
df.loc[df['Total Volume'].isna()]

,Date,Price,Total Volume,Total Boxes,Small Boxes,Large Boxes,XLarge Boxes,Region
38,2018-03-11,NaN,NaN,2422.18,664.03,1758.15,0.0,deagu


In [8]:
df[df.duplicated()]

,Date,Price,Total Volume,Total Boxes,Small Boxes,Large Boxes,XLarge Boxes,Region


In [9]:
def date_to_datetime(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def filter(df, cond):
    return df.loc[cond]

def identity(df):
    return df

def extract_date_price_region(df):
    return df[['Date', 'Price', 'Region']]

In [10]:
def find(fp, price, date, region):
    
    filter_price = partial(filter, cond=lambda df: df['Price'] == price) if price else identity
    filter_date = partial(filter, cond=lambda df: df['Date'] == date) if date else identity
    filter_region = partial(filter, cond=lambda df: df['Region'] == region) if region else identity

    df = (pd.read_excel(fp)
            .dropna(how='any', axis=0)
            .pipe(date_to_datetime, 'Date')
            .pipe(filter_price)
            .pipe(filter_date)
            .pipe(filter_region)
            .pipe(extract_date_price_region)
        )
    
    return df.sort_values(by=['Date'])

In [11]:
res = find('Kimchi_dataset.xlsx', 1.45, '2018-01-14', 'Chungju')
res

,Date,Price,Region
202,2018-01-14,1.45,Chungju


In [12]:
res = find('Kimchi_dataset.xlsx', None, '2018-01-14', 'Chungju')
res

,Date,Price,Region
202,2018-01-14,1.45,Chungju
214,2018-01-14,2.02,Chungju
622,2018-01-14,1.59,Chungju
634,2018-01-14,1.82,Chungju


In [13]:
res = find('Kimchi_dataset.xlsx', 1.45, '2018-01-14', None)
res

,Date,Price,Region
94,2018-01-14,1.45,Busan
190,2018-01-14,1.45,Boryeong
202,2018-01-14,1.45,Chungju


In [14]:
res = find('Kimchi_dataset.xlsx', 1.45, None, None)
res

,Date,Price,Region
143,2018-01-07,1.45,Gangeung
94,2018-01-14,1.45,Busan
190,2018-01-14,1.45,Boryeong
202,2018-01-14,1.45,Chungju
140,2018-01-28,1.45,Gangeung
224,2018-01-28,1.45,Geoje
187,2018-02-04,1.45,Boryeong
185,2018-02-18,1.45,Boryeong
461,2018-02-18,1.45,Yangju
160,2018-02-25,1.45,Andong


In [15]:
res = find('Kimchi_dataset.xlsx', 1.45, None, 'Boryeong')
res

,Date,Price,Region
190,2018-01-14,1.45,Boryeong
187,2018-02-04,1.45,Boryeong
185,2018-02-18,1.45,Boryeong


In [16]:
res = find('Kimchi_dataset.xlsx', None, None, 'Boryeong')
res

,Date,Price,Region
191,2018-01-07,1.40,Boryeong
647,2018-01-07,1.62,Boryeong
646,2018-01-14,1.93,Boryeong
190,2018-01-14,1.45,Boryeong
189,2018-01-21,1.41,Boryeong
645,2018-01-21,1.87,Boryeong
188,2018-01-28,1.42,Boryeong
644,2018-01-28,1.71,Boryeong
187,2018-02-04,1.45,Boryeong
643,2018-02-04,1.63,Boryeong


In [17]:
res = find('Kimchi_dataset.xlsx', None, None, None)
res

,Date,Price,Region
323,2018-01-07,1.57,Taebaek
335,2018-01-07,1.43,Pyeongtaek
311,2018-01-07,1.22,Taebaek
299,2018-01-07,1.47,Suwon
287,2018-01-07,1.38,Jeonju
...,...,...,...
360,2018-03-25,1.63,Sangju
24,2018-03-25,1.33,Incheon
336,2018-03-25,1.86,Yeoju
312,2018-03-25,1.48,Taebaek
